# Graphing Inputs/Outputs of Pre and Post-Synaptic Neurons

## Import all relevent libraries

In [ ]:
#Import all relevent libraries
from caveclient import CAVEclient
import nglui
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from phase3 import nda

## Using CAVEclient to query data

In [ ]:
client = CAVEclient()

In [ ]:
client.auth.save_token(token="a80f7690d93ead1b46e980628df5beae",overwrite = True)

In [ ]:
# this is the datastack name of the public release
# passing it will auto-configure many of the services
client = CAVEclient('minnie65_public_v343')

In [ ]:
client.materialize.get_tables()

## Filter Data to Obtain Neurons w/ Functional Co-registration

In [ ]:
df_fun_coreg=client.materialize.query_table('functional_coreg') #dataframe of the neurons that have functional data available

In [ ]:
df_fun_coreg.head() #visualize first couple neurons

In [ ]:
seg_ids = df_fun_coreg.iloc[:, 3] #extract column of IDs to loop over
seg_ids

## Loop Over Neurons to Plot Input/Output Depth

In [ ]:
 #for loop so we can obtain the depth values for each presynaptic neuron
for id in seg_ids[:30]:
    seg_id = id #define a single neuron to look at pre/post synaptic connections
    input_df = client.materialize.synapse_query(post_ids=seg_id) #define inputs (presynaptic neurons) of the cell
    input_df.rename(columns = {'pre_pt_root_id':'pt_root_id'}, inplace = True) #rename presynaptic id column so we can inner join with df_fun_coreg
    fun_coreg_matched_df = pd.merge(input_df,df_fun_coreg,on='pt_root_id',how='inner') #merge the tables to filter for presynaptic connections that are also in the functional set
    
    #this for loop allows us to get the "scan keys" for each of the neurons in the "fun_coreg_matched_df" table.
    scan_keys = []
    dict1 = {}
    for i in range(len(fun_coreg_matched_df)):
        dict1 = {'session': fun_coreg_matched_df.loc[i].at['session'], 'scan_idx': fun_coreg_matched_df.loc[i].at['scan_idx'], 'unit_id': fun_coreg_matched_df.loc[i].at['unit_id']}
        scan_keys.append(dict1)
      
    #using the scan_keys, we can obtain the depth value for each of the presynaptic connections to the current neuron in the loop. This will be graph on the X-Axis as "Input Depth"
    graph_x_values= np.stack((nda.StackUnit() & scan_keys).fetch('stack_z'))
    
    #this if statement checks if the list of presynaptic neuron depths is an empty set. If it is empty, we end the current iteration
    if graph_x_values.any() == False:
        continue
        
    #The next 2 lines get the depth of the current neuron. This will be graphed on the Y-Axis as "Output Depth
    f= nda.StackUnit() & df_fun_coreg.loc[df_fun_coreg["pt_root_id"]==id]
    graph_y_value = np.stack((f).fetch("stack_z"), -1)
    
    #Because "graph_y_value" is the depth of a single neuron, there should be only a singular value in the list. However, we have seen some anomolies where there are multiple depths listed
    #Until we figure out why this occurs, we simply filter out these neurons. The amount of anomolies relative to the size of the whole set is insignificant so this does not affect our output.
    if len(graph_y_value) != 1:
        continue
        
    #Graph the current neuron depth as Output Depth and all it's presynaptic cell depths as input depths. Each iteration will usually result in multiple datapoints on the plot
    for ye in graph_x_values:
        plt.scatter(ye, graph_y_value, s=1)
    
    
    
    
    
    